In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

from bs4 import BeautifulSoup
import time
import folium
import collections
from collections import OrderedDict
import csv



In [5]:
chrome_driver = '../drv/chromedriver.exe'
driver = webdriver.Chrome(chrome_driver)
driver.implicitly_wait(5) # 응답의 시간 지연
# url = 'https://www.starbucks.co.kr/store/getStore.do?r=24F3U9PI7Q'
url = 'https://www.starbucks.co.kr/store/store_map.do'
# chrome driver로 해당 페이지가 물리적으로 open
driver.get(url)

# 값을 담을 리스트
starbucks = []

# 열린 페이지에서 '지역 검색' 탭 클릭
search = driver.find_element_by_link_text('지역 검색')
time.sleep(1)
search.click()

time.sleep(1)
# 개발자도구로 class : set_sido_cd_btn의 데이터 긁어옴
search = driver.find_elements_by_class_name('set_sido_cd_btn') # 지역

for item in search:
    item.click()
    time.sleep(1)

    # data-sidocd='01~17' 서울~세종
    if '17' == item.get_attribute('data-sidocd'): # 지역에 부여된 번호
        # 소스 가져오기
        src = driver.page_source

        # BeautifulSoup 객체로 변환
        soup = BeautifulSoup(src)
        name = soup.select('li[data-name]')
        for name_one in name:
            x = name_one['data-lat'] # 위도 저장
            y = name_one['data-long'] # 경도 저장
            z = name_one['data-name'] # 지점명 저장
            p = name_one.select_one('p').text.split('1522-3232')[0] # 번호는 모든 지점이 동일하여 crawling에서 제외
            starbucks.append({'name': z, 'address': p, 'lat': x, 'long':y}) # dict 형태로 리스트에 저장
        time.sleep(1)

        # 열린 페이지 닫기
        driver.close()
    else:
        search2 = driver.find_element_by_link_text('전체')
        search2.click()
        driver.implicitly_wait(3) # 셀레니움 자체 대기시간 부여
        time.sleep(1)

        src = driver.page_source

        soup = BeautifulSoup(src)
        
        name = soup.select('li[data-name]') # li 안의 data-name 이 있는데 그게 지역가게 명
        
        for name_one in name:
            x = name_one['data-lat'] # 경도
            y = name_one['data-long'] #위도
            z = name_one['data-name']
            p = name_one.select_one('p').text.split('1522-3232')[0]
            starbucks.append({'name': z, 'address': p, 'lat': x, 'long':y})
        time.sleep(1)

        # 다시 지역 검색 탭으로 돌아가기위한 소스
        search3 = driver.find_element_by_link_text('지역 검색')
        search3.click()
        time.sleep(1)

<ipython-input-5-eff0d6a85c26>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chrome_driver)
<ipython-input-5-eff0d6a85c26>:13: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  search = driver.find_element_by_link_text('지역 검색')
<ipython-input-5-eff0d6a85c26>:19: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  search = driver.find_elements_by_class_name('set_sido_cd_btn') # 지역
<ipython-input-5-eff0d6a85c26>:44: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  search2 = driver.find_element_by_link_text('전체')
<ipython-input-5-eff0d6a85c26>:64: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  search3 = driver.find_element_by_link_text('지역 검색')


WebDriverException: Message: chrome not reachable
  (Session info: chrome=95.0.4638.69)
Stacktrace:
Backtrace:
	Ordinal0 [0x00F106F3+2492147]
	Ordinal0 [0x00EA9BD1+2071505]
	Ordinal0 [0x00DB2340+1057600]
	Ordinal0 [0x00DA7474+1012852]
	Ordinal0 [0x00DA7C76+1014902]
	Ordinal0 [0x00DA9345+1020741]
	Ordinal0 [0x00DA3453+996435]
	Ordinal0 [0x00DB3690+1062544]
	Ordinal0 [0x00E046D2+1394386]
	Ordinal0 [0x00DF568B+1332875]
	Ordinal0 [0x00DD21D4+1188308]
	Ordinal0 [0x00DD302F+1191983]
	GetHandleVerifier [0x010967A6+1545030]
	GetHandleVerifier [0x0114105C+2243580]
	GetHandleVerifier [0x00F9BC97+518199]
	GetHandleVerifier [0x00F9AD80+514336]
	Ordinal0 [0x00EAED2D+2092333]
	Ordinal0 [0x00EB2EE8+2109160]
	Ordinal0 [0x00EB3022+2109474]
	Ordinal0 [0x00EBCB71+2149233]
	BaseThreadInitThunk [0x76ADFA29+25]
	RtlGetAppContainerNamedObjectPath [0x77B17A9E+286]
	RtlGetAppContainerNamedObjectPath [0x77B17A6E+238]


In [3]:
# 지도 작성
map_list = folium.Map(location=[35.95, 128.25], zoom_start=7)

for item in starbucks:
    latitude = float(item['lat']) # 위도
    longtitude = float(item['long']) # 경도
    # 지점 위치 표시
    folium.Marker(
        location = [latitude, longtitude],
        popup = 'jijum : {}, addr : {}'.format(item['name'], item['address']),
        # icon 적용 할 때 부트스트랩 최신버전을 사용하면 표시안됨. 3버전 이용
        icon=folium.Icon(color='green', icon='glyphicon glyphicon-star')
    ).add_to(map_list)
map_list

# 저장

In [6]:
# newline=''을 넣으면 공백라인 제거
with open('starbucks.csv', 'w', newline='') as f:
    fieldnames =['지점명', '주소', '위도', '경도']
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    for i in starbucks:
        writer.writerow({'지점명': i['name'], '주소': i['address'], '위도': i['lat'], '경도': i['long']})